In [1]:
import os.path
import pandapower.networks as pn
import pandapower as pp
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from numpy import flatnonzero as find, ones, zeros, Inf, pi, exp, conj, r_, arange, array
from pypower_.makeYbus import makeYbus
from pypower_.idx_brch import F_BUS, T_BUS, RATE_A, PF, QF, PT, QT, MU_SF, MU_ST
from pypower_.idx_gen import GEN_BUS, PG, QG
from pypower_.idx_cost import MODEL, PW_LINEAR, NCOST, POLYNOMIAL
from pypower_.totcost import totcost
from pypower_.makeSbus import makeSbus
from pypower_.idx_bus import BUS_TYPE, REF, VM, VA, MU_VMAX, MU_VMIN, LAM_P, LAM_Q
import cvxpy as cp
import numpy as np
from scipy.sparse import csr_matrix as sparse
from pypower_.idx_bus import PD, QD
from pypower_.idx_gen import GEN_BUS, PG, QG, GEN_STATUS

/home/huzuntao/PycharmProjects/MPP_Powersystem/venv/lib/python3.8/site-packages/numba/core/errors.py:175: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
import sys
print(sys.path)
print(os.path)
path_current = '/home/ubuntu-h/PycharmProjects/scientificProject'
if path_current not in sys.path:
    sys.path.insert(1, '/home/ubuntu-h/PycharmProjects/scientificProject')

['/home/huzuntao/PycharmProjects/MPP_Powersystem/notebooks', '/home/huzuntao/PycharmProjects/MPP_Powersystem', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/huzuntao/PycharmProjects/MPP_Powersystem/venv/lib/python3.8/site-packages', '/usr/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']
<module 'posixpath' from '/usr/lib/python3.8/posixpath.py'>


In [3]:
import pandapower as pp
net = pp.create_empty_network()

# # create buses
# bus1 = pp.create_bus(net, vn_kv=110.)
# bus2 = pp.create_bus(net, vn_kv=110.)
# bus3 = pp.create_bus(net, vn_kv=110.)
# # bus4 = pp.create_bus(net, vn_kv=110.)
# # bus5 = pp.create_bus(net, vn_kv=110.)
#
# # create 110 kV lines
# pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
# # pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
# # pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")
# pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')
#
# # create loads
# pp.create_load(net, bus1, p_mw=30., controllable=False)
# # pp.create_load(net, bus3, p_mw=70., controllable=False)
# # pp.create_load(net, bus4, p_mw=25., controllable=False)
#
# # create generators
# eg = pp.create_ext_grid(net, bus3, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
# g0 = pp.create_gen(net, bus2, p_mw=80, min_p_mw=0, max_p_mw=50, vm_pu=1.00, controllable=True)
# # g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.00, controllable=True)
#
#
# costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
# costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
# costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
# bus4 = pp.create_bus(net, vn_kv=110.)
# bus5 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=90., std_type='149-AL1/24-ST1A 110.0')
# pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
# pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
pp.create_load(net, bus2, p_mw=50., controllable=False)
# pp.create_load(net, bus3, p_mw=70., controllable=False)
# pp.create_load(net, bus4, p_mw=25., controllable=False)

# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=50, min_p_mw=0, max_p_mw=50, vm_pu=1.00, controllable=True)
# g1 = pp.create_gen(net, bus4, p_mw=50, min_p_mw=0, max_p_mw=50, vm_pu=1.00, controllable=True)


costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.bus["min_vm_pu"] = 0.96
net.bus["max_vm_pu"] = 1.04
net.line["max_loading_percent"] = 100

![xx -w50](grid2GUROBI.png)

In [4]:
net.line["max_loading_percent"] = 100
# net.sn_mva = 100
# net = pp.networks.case14()
# net.gen['vm_pu'] = 1.0
# net.bus["min_vm_pu"] = 0.95
# net.bus["max_vm_pu"] = 1.05
# net.poly_cost[:,3] = 0
# net.poly_cost['cp2_eur_per_mw2'] = 0
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,10.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,10.0,0.0,0.0,0.0,0.0


In [5]:
om,ppopt = pp.runopp(net, delta=1e-16)   # solve the OPF problem

In [6]:
# net = pp.networks.case57()
print(net.res_gen)    # output the results of two generators
net.gen

        p_mw   q_mvar  va_degree  vm_pu
0  49.999981 -11.4693   9.541393   1.04


,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,min_p_mw,max_p_mw,power_station_trafo
0,None,2,50.0,1.0,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,50.0,NaN


In [7]:
print(net.res_ext_grid)
net.ext_grid['vm_pu']

       p_mw     q_mvar
0  3.706462  13.673431


0    1.05
Name: vm_pu, dtype: float64

In [8]:
# the total cost of generation
net.res_cost

574.1290514078155

In [9]:
# the key variable, which represents the information of the OPF problem
om.get_ppc()

{'baseMVA': 1,
 'version': 2,
 'bus': array([[ 0.00000000e+00,  3.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.05000000e+00,  0.00000000e+00,
          1.10000000e+02,  1.00000000e+00,  1.05000000e+00,
          1.05000000e+00,  1.99999635e+01,  3.70893752e-21,
          0.00000000e+00,  2.11577135e+02],
        [ 1.00000000e+00,  1.00000000e+00,  5.00000000e+01,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.01227994e+00,  4.30977831e-01,
          1.10000000e+02,  1.00000000e+00,  1.04000000e+00,
          9.60000000e-01,  1.99209142e+01,  1.35379538e-01,
          0.00000000e+00,  0.00000000e+00],
        [ 2.00000000e+00,  2.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00,  1.03999961e+00,  9.54139328e+00,
          1.10000000e+02,  1.00000000e+00,  1.04000000e+00,
          9.60000000e-01,  1.703128

In [10]:
## unpack data
ppc = om.get_ppc()
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]
vv, _, _, _ = om.get_idx()

## problem dimensions
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches
ny = om.getN('var', 'y')   ## number of piece-wise linear costs, 'y' is a part of p-w linear costs

## bounds on optimization vars
x0, xmin, xmax = om.getv()

In [11]:
## build admittance matrices
Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

/home/huzuntao/PycharmProjects/MPP_Powersystem/pypower_/makeYbus.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  tap[i] = branch[i, TAP]  ## assign non-zero tap ratios
/usr/local/lib/python3.8/dist-packages/scipy/sparse/_coo.py:159: ComplexWarning: Casting complex values to real discards the imaginary part
  self.col = np.array(col, copy=copy, dtype=idx_dtype)


In [12]:
## Set the lower and upper bound for all variables
ll, uu = xmin.copy(), xmax.copy()
# print(f'll : {ll}; uu : {uu}')
ll[xmin == -Inf] = -1e10   ## replace Inf with numerical proxies
uu[xmax ==  Inf] =  1e10
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
# print(f"Va_refs: {Va_refs}")
ll[vv["i1"]["Va"]:vv["iN"]["Va"]] = -np.ones_like(bus[:, VA]) * (pi / 2) # Va lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]] = np.ones_like(bus[:, VA]) * (pi / 2) # Va upper bound 赋值
## deal with the Va_refs
ll[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs  # Va_refs lower bound 赋值
uu[vv["i1"]["Va"]:vv["iN"]["Va"]][bus[:, BUS_TYPE]  == REF] = Va_refs # Va_refs upper bound 赋值
## deal with reactive power, whose ll and uu is 1e9
ll[ll<-1e4] = -100
uu[uu>1e4] = 100
v_max = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]][-1]
print(f'll"{ll};\n uu:{uu}')

ll"[ 0.00000000e+00 -1.57079633e+00 -1.57079633e+00  1.05000000e+00
  9.60000000e-01  9.60000000e-01 -1.00000000e-16 -1.00000000e-16
 -1.00000000e+02 -1.00000000e+02];
 uu:[   0.            1.57079633    1.57079633    1.05          1.04
    1.04       1000.           50.          100.          100.        ]


In [13]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]

array([1.05, 1.  , 1.  ])

In [14]:
uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]

array([1.05, 1.04, 1.04])

In [15]:
bus[:, VM] # 赋值

array([1.05      , 1.01227994, 1.03999961])

In [16]:
x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]] = bus[:, VM] # 赋值

In [17]:
Va_refs = bus[bus[:, BUS_TYPE]  == REF, VA] * (pi / 180)
x0[vv["i1"]["Va"]:vv["iN"]["Va"]] = bus[:, VA] * (pi / 180) # 赋值

In [18]:
## 赋值
x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]] = gen[:,PG]
x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]] = gen[:,QG]


### 不能处理piecewise linear 的cost, 因为没有引入那个函数，（大约在这个位置）


### 处理cost fun

In [19]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

##----- evaluate objective function -----
## polynomial cost of P and Q
# use totcost only on polynomial cost in the minimization problem formulation, pwl cost is the sum of the y variables.
ipol = find(gencost[:, MODEL] == POLYNOMIAL)   ## poly MW and MVAr costs
xx = r_[ Pg, Qg ] * baseMVA
if len(ipol)>0:
    f = sum( totcost(gencost[ipol, :], xx[ipol]) )  ## cost of poly P or Q
else:
    f = 0

In [20]:
Pg

array([ 3.70646221, 49.99998071])

In [21]:
xx[ipol]

array([ 3.70646221, 49.99998071])

In [22]:
First_Or_Con = 4
print(f"{gencost[ipol, First_Or_Con], gencost[:, First_Or_Con]}")

(array([20., 10.]), array([20., 10.]))


In [23]:
gencost

array([[ 2.,  0.,  0.,  2., 20.,  0.],
       [ 2.,  0.,  0.,  2., 10.,  0.]])

### 处理约束部分

In [24]:
## grab Pg & Qg
Pg = x0[vv["i1"]["Pg"]:vv["iN"]["Pg"]]  ## active generation in p.u.
Qg = x0[vv["i1"]["Qg"]:vv["iN"]["Qg"]]  ## reactive generation in p.u.

In [25]:
## put Pg & Qg back in gen
gen[:, PG] = Pg * baseMVA  ## active generation in MW
gen[:, QG] = Qg * baseMVA  ## reactive generation in MVAr

In [26]:

on = find(gen[:, GEN_STATUS] > 0)      ## which generators are on?
gbus = gen[on, GEN_BUS]                   ## what buses are they at?

## form net complex bus power injection vector
nb = bus.shape[0]
ngon = on.shape[0]
## connection matrix, element i, j is 1 if gen on(j) at bus i is ON
Cg = sparse((ones(ngon), (gbus, range(ngon))), (nb, ngon))

## power injected by gens plus power injected by loads converted to p.u.
Sbus = ( Cg * (gen[on, PG] + 1j * gen[on, QG]) - (bus[:, PD] + 1j * bus[:, QD]) ) / baseMVA
# Sbus = makeSbus(baseMVA, bus, gen) ## net injected power in p.u.

In [27]:
Sbus

array([  3.70646221+13.67343146j, -50.         +0.j        ,
        49.99998071-11.46930012j])

In [28]:
## reconstruct V
Va = x0[vv["i1"]["Va"]:vv["iN"]["Va"]]
Vm = x0[vv["i1"]["Vm"]:vv["iN"]["Vm"]]
V = Vm * exp(1j * Va)
V

array([1.05      +0.j        , 1.0122513 +0.00761428j,
       1.02561237+0.17239044j])

In [29]:
Va

array([0.        , 0.00752198, 0.16652873])

In [30]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.05000,0.000000,-3.706462,-13.673431,19.999964,3.708938e-21
1,1.01228,0.430978,50.000000,0.000000,19.920914,1.353795e-01
2,1.04000,9.541393,-49.999981,11.469300,17.031287,-3.111046e-21


In [31]:
## evaluate power flow equations
mis = V * conj(Ybus * V) - Sbus

In [32]:
ppc['branch']

array([[ 1.00000000e+00+0.j,  2.00000000e+00+0.j,  1.44297521e-03+0.j,
         3.04958678e-03+0.j,  2.99354510e+00+0.j,  8.95470268e+01+0.j,
         2.50000000e+02+0.j,  2.50000000e+02+0.j,  1.00000000e+00+0.j,
         0.00000000e+00+0.j,  1.00000000e+00+0.j, -3.60000000e+02+0.j,
         3.60000000e+02+0.j, -4.65352523e+01+0.j,  1.56389975e+01+0.j,
         4.99999807e+01+0.j, -1.14693001e+01+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  1.00000000e+00+0.j,  1.12231405e-03+0.j,
         2.37190083e-03+0.j,  2.32831286e+00+0.j,  8.95470268e+01+0.j,
         2.50000000e+02+0.j,  2.50000000e+02+0.j,  1.00000000e+00+0.j,
         0.00000000e+00+0.j,  1.00000000e+00+0.j, -3.60000000e+02+0.j,
         3.60000000e+02+0.j,  3.70646222e+00+0.j,  1.36734315e+01+0.j,
        -3.46474770e+00+0.j, -1.56389975e+01+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.

In [33]:
branch[:, RATE_A]

array([89.54702675+0.j, 89.54702675+0.j])

In [34]:
 ## find branches with flow limits
il = find((branch[:, RATE_A] != 0) & (branch[:, RATE_A] < 1e10))
nl2 = len(il)           ## number of constrained lines

In [35]:
il

array([0, 1])

In [36]:
## 不等式约束： 线电压的功率
ppopt = {}
il = find(branch[:, RATE_A] < 1e10)
nl2 = len(il)           ## number of constrained lines
if any(net.line['max_i_ka']) > 0:
    ppopt['OPF_FLOW_LIM'] = 2

if nl2 > 0:
    flow_max = (branch[il, RATE_A] / baseMVA)**2
    flow_max[flow_max == 0] = 1e5
    print(f"flow_max: {flow_max}")
    if ppopt['OPF_FLOW_LIM'] == 2:      ## current magnitude limit, |I|
        If = Yf * V
        It = Yt * V
        h = r_[ If * conj(If) - flow_max,     ## branch I limits (from bus)
                It * conj(It) - flow_max ].real    ## branch I limits (to bus)
    else:
        ## compute branch power flows
        ## complex power injected at "from" bus (p.u.)
        Sf = V[ branch[il, F_BUS].astype(int) ] * conj(Yf * V)
        ## complex power injected at "to" bus (p.u.)
        St = V[ branch[il, T_BUS].astype(int) ] * conj(Yt * V)
        if ppopt['OPF_FLOW_LIM'] == 1:   ## active power limit, P (Pan Wei)
            h = r_[ Sf.real**2 - flow_max,   ## branch P limits (from bus)
                    St.real**2 - flow_max ]  ## branch P limits (to bus)
        else:                ## apparent power limit, |S|
            h = r_[ Sf * conj(Sf) - flow_max, ## branch S limits (from bus)
                    St * conj(St) - flow_max ].real  ## branch S limits (to bus)
else:
    h = zeros((0,1))

flow_max: [8018.67+0.j 8018.67+0.j]


In [37]:
## first, the equality constraints (power flow)
# g = r_[ mis.real,            ## active power mismatch for all buses
#         mis.imag ]           ## reactive power mismatch for all buses

## then, the inequality constraints (branch flow limits)
h

array([-5666.6812893 , -7836.62864855, -5585.65882681, -7768.27471867])

In [38]:
If

array([-45.85322576-15.79463194j,   3.52996401-13.02231567j])

In [39]:
flow_max

array([8018.67+0.j, 8018.67+0.j])

In [40]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,-46.535252,15.638997,49.999981,-11.469300,3.464728,4.169697,0.254545,0.258892,0.258892,1.01228,0.430978,1.04000,9.541393,55.083418
1,3.706462,13.673431,-3.464748,-15.638997,0.241715,-1.965566,0.070816,0.083054,0.083054,1.05000,0.000000,1.01228,0.430978,17.671031


In [41]:
llv = ll[vv["i1"]["Vm"]:vv["iN"]["Vm"]]**2
uuv = uu[vv["i1"]["Vm"]:vv["iN"]["Vm"]]**2
uuv

array([1.1025, 1.0816, 1.0816])

In [42]:
print(Ybus)

  (0, 0)	(162.99668368338897-343.3133709444148j)
  (1, 0)	(-162.99668368338897+344.47752737211067j)
  (0, 1)	(-162.99668368338897+344.47752737211067j)
  (1, 1)	(289.77188210380257-609.743564128384j)
  (2, 1)	(-126.77519842041363+267.9269657338639j)
  (1, 2)	(-126.77519842041363+267.9269657338639j)
  (2, 2)	(126.77519842041363-266.4301931839692j)


In [43]:
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]
Pg_upper

array([1000.,   50.])

In [44]:
Pd = bus[:, PD]
Pd

array([ 0., 50.,  0.])

In [45]:
baseMVA

1

In [46]:
# Parameters
# obtain the complex load
Pd = bus[:, PD]
Qd = bus[:, QD]
slack_v = net.ext_grid['vm_pu']

lb_update = -1e4

# obtain the range of voltage; the range is from the setting
V_range = v_max - 1
V_ll,V_uu = 1.0-V_range, 1.0+V_range

# obtain the power range
Pg_lower, Pg_upper = ll[vv["i1"]["Pg"]:vv["iN"]["Pg"]], uu[vv["i1"]["Pg"]:vv["iN"]["Pg"]]*baseMVA
Qg_lower, Qg_upper = ll[vv["i1"]["Qg"]:vv["iN"]["Qg"]], uu[vv["i1"]["Qg"]:vv["iN"]["Qg"]]*baseMVA

# the dimension of the OPF
nb = bus.shape[0]          ## number of buses
nl = branch.shape[0]       ## number of branches

ineq_idx = 0

# create the model; set the model type: NonConvex.
model = gp.Model("PowerGeneration");model.setParam('NonConvex', 2)

# creat variables for the model; the numbers in the following codes should be replaced by parameters
V_re = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V real')
V_im = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='V imag')
Pg = model.addMVar(ngon, lb=Pg_lower, ub=Pg_upper, vtype=GRB.CONTINUOUS, name='Pg')
Qg = model.addMVar(ngon, lb=Qg_lower, ub=Qg_upper, vtype=GRB.CONTINUOUS, name='Qg')
temp1 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 1')
temp2 = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Temp 2')
Pg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Pg net')  # the number of buses, too
Qg_net = model.addMVar(nb, lb=lb_update, vtype=GRB.CONTINUOUS, name='Qg net')
Line_temp_1 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 1')    # the shape should be line number
Line_temp_2 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 2')    # the shape should be line number
Line_temp_3 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 3')    # the shape should be line number
Line_temp_4 = model.addMVar(nl, lb=lb_update, vtype=GRB.CONTINUOUS, name='Line temp 4')    # the shape should be line number

# the prameters of branches
G = np.real(Ybus)
B = np.imag(Ybus)
# the parameters for currents in branches
G_yf, B_yf = np.real(Yf), np.imag(Yf)
G_yt, B_yt = np.real(Yt), np.imag(Yt)

# Constraints; insert the formulas
Temp_c1 = model.addConstr(G@V_re - B@V_im == temp1, 'Temp Constr 1')
Temp_c2 = model.addConstr(G @ V_im + B @ V_re == temp2, 'Temp Constr 2')
Pg_net_c = model.addConstr(Cg @ Pg - Pd == Pg_net, 'Pg net Constr')
Qg_net_c = model.addConstr(Cg @ Qg - Qd == Qg_net, 'Qg net Constr')
# print(f'Pd:{Pd}\n Qd:{Qd}')
CVL_CPL = model.addConstrs((V_re[i] * temp1[i] *baseMVA+ V_im[i] * temp2[i]*baseMVA == Pg_net[i]  for i in range(nb)), 'Cvl P')  # break into different parts
CVL_CPL_Q = model.addConstrs((V_im[i] * temp1[i] *baseMVA - V_re[i] * temp2[i]*baseMVA == Qg_net[i] for i in range(nb)), 'Cvl Q')

# the constraints of voltages on all buses; set the constraint for slack bus, then give the upper and lower constraints for other buses
V_slack = model.addConstr(V_re[0]==slack_v, 'V slack real')
V_slack_a = model.addConstr(V_im[0]==0, 'V slack imag')
# the upper&lower constraints for buses except for the slack bus
V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 <= V_uu**2 for i in range(1,nb-ineq_idx)), 'V upper bound')  # 对 range 起始值的处理，是处理树状分层搜索的突破口；有一"减"，对应的有一"加"。
# if ineq_idx != 0:
#     V_upper = model.addConstrs((V_re[i]**2 + V_im[i]**2 == V_uu**2 for i in range(ineq_idx,nb)), 'V upper bound')
#
V_lower = model.addConstrs((V_re[i]**2 + V_im[i]**2 >= V_ll**2 for i in range(1,nb)), 'V lower bound')

# Yf * W * Yf; the number is equal to that of the lines/branches
Line_temp_c_1 = model.addConstr(G_yf@V_re - B_yf@V_im == Line_temp_1, 'Branch Con 1')
Line_temp_c_2 = model.addConstr(G_yf @ V_im + B_yf @ V_re == Line_temp_2, 'Branch Con 2')
Line_temp_c_3 = model.addConstr(G_yt@V_re - B_yt@V_im == Line_temp_3, 'Branch Con 3')
Line_temp_c_4 = model.addConstr(G_yt @ V_im + B_yt @ V_re == Line_temp_4, 'Branch Con 4')
# 下式左侧是否应该乘上baseMVA
Line_c_1 = model.addConstrs((Line_temp_1[i]**2 + Line_temp_2[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 1') # p.u.
Line_c_2 = model.addConstrs((Line_temp_3[i]**2 + Line_temp_4[i]**2 <= np.real(flow_max[i]) for i in range(nl)), 'Branch Con 2') # p.u.

# set the objective of the model
print('ipol:', ipol)
# The "gencost" changes with different cases. Keep in Mind!
# model.setObjective(gencost[ipol, First_Or_Con]@Pg**2+gencost[ipol, First_Or_Con+1]@Pg, GRB.MINIMIZE)
model.setObjective(gencost[ipol, First_Or_Con]@Pg, GRB.MINIMIZE)

# calculate the optimization
model.optimize()
model.X[:14]

Pg_star, Qg_star = Pg.X, Qg.X
optimized_obj = gencost[ipol, First_Or_Con]@Pg_star


Set parameter NonConvex to value 2
ipol: [0 1]
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 22 rows, 30 columns and 86 nonzeros
Model fingerprint: 0x03c14a1c
Model has 14 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [1e-16, 1e+04]
  RHS range        [1e+00, 5e+01]
  QRHS range       [9e-01, 8e+03]
Presolve removed 4 rows and 4 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 12 rows and 14 columns
Presolve time: 0.00s
Presolved: 52 rows, 28 columns, 160 nonzeros
Presolved model has 6 quadratic constraint(s)
Presolved model has 12 bilinear constraint(s)
Variable types: 28 continuous, 0 integer (0 bin

In [47]:
len(model.getVars())
len(model.getConstrs())

22

In [48]:
print(Pg)

<MVar (2,)>
array([<gurobi.Var Pg[0] (value 3.706439003337366)>,
       <gurobi.Var Pg[1] (value 49.999999999765286)>])


In [49]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.05000,0.000000,-3.706462,-13.673431,19.999964,3.708938e-21
1,1.01228,0.430978,50.000000,0.000000,19.920914,1.353795e-01
2,1.04000,9.541393,-49.999981,11.469300,17.031287,-3.111046e-21


In [50]:
m = gp.Model("model")
# x = m.addMVar(shape=(10), name="x")
y = m.addMVar(shape=(5),lb=np.array([-1,-2,-3,-4,-5]), name="y")

A = np.ones(5)

m.addConstrs((y[i] >=np.array([-1,-2,-3,-4,-5])[i] for i in range(5)), name="const")
m.setObjective(A@Y_.real)
m.optimize()

NameError: name 'Y_' is not defined

In [ ]:
m.X

In [ ]:
for v in m.getVars():
    print(v.varName, v.x)

In [ ]:
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

print(f'A ;{A};\n b : {b}')

In [ ]:
# Define and solve the CVXPY problem.
# Create a symmetric matrix variable
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve()
# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)